# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [113]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle
import sympy as sp

In [10]:
%matplotlib tk

# Provided functions

In [11]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.axis('off')
    plt.imshow(im)
    print(f'Set at least {min_lines} lines to compute vanishing point')
    print(f'The delete and backspace keys act like right clicking')
    print(f'The enter key acts like middle clicking')
    while True:
        print('Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print(f'Need at least {min_lines} lines, you have {n} now')
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [12]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [50]:
def get_vanishing_point(ls):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR CODE>
    is1 = np.cross(ls[:, 0], ls[:, 1])
    is2 = np.cross(ls[:, 1], ls[:, 2])
    is3 = np.cross(ls[:, 0], ls[:, 2])
    
    iss = np.vstack((is1/is1[-1], is2/is2[-1], is3/is3[-1]))
    vpt = np.mean(iss, axis=0)
    return vpt

In [104]:
def get_horizon_line(vpt1, vpt2):
    """
    Calculates the ground horizon line.
    """
    # <YOUR CODE>
    a, b, c = np.cross(vpt1, vpt2)
    scale = np.sqrt(1 / (a**2 + b**2))
    line = np.array([a, b, c]) * scale
    return line
# get_horizon_line(vpts)

In [107]:
def plot_horizon_line(ax, im, vpt1, vpt2):
    """
    Plots the horizon line.
    """
    # <YOUR CODE>
    ax.imshow(im)
    ax.plot([vpt1[0], vpt2[0]], [vpt1[1], vpt2[1]], 'g')
    return

In [145]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR CODE>
    vpt1 = sp.Matrix(vpts[:, 0]) 
    vpt2 = sp.Matrix(vpts[:, 1]) 
    vpt3 = sp.Matrix(vpts[:, 2])
#     print(p1, p2, p3)
    k = sp.Matrix((('f', 0, 'u'), (0, 'f', 'v'), (0, 0, 1)))
    kinv = k.inv()
    f, u, v = sp.solve([vpt1.T*kinv.T*kinv*vpt2, 
                 vpt1.T*kinv.T*kinv*vpt3, 
                 vpt2.T*kinv.T*kinv*vpt3], 
                ['f', 'u', 'v'])[0]
    
    k = np.array([[f, 0, u],
                 [0, f, v],
                 [0, 0, 1]], dtype = 'float')
    return f, u, v, k
f, u, v, k = get_camera_parameters(vpts)
print(f, u, v, k)

-2334.13539207586 2048.06544224965 1120.52389673485 [[-2.33413539e+03  0.00000000e+00  2.04806544e+03]
 [ 0.00000000e+00 -2.33413539e+03  1.12052390e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [157]:
def get_rotation_matrix(vpts, f, k):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR CODE>
    vpt1 = vpts[:, 2]
    vpt2 = vpts[:, 1]
    vpt3 = vpts[:, 0]
#     print(k)
    kinv = np.linalg.inv(k)
    r1 = np.dot(kinv, vpt1)
    r2 = np.dot(kinv, vpt2)
    r3 = np.dot(kinv, vpt3)
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    r1 = r1.reshape((3,1))
    r2 = r2.reshape((3,1))
    r3 = r3.reshape((3,1))
    R = np.hstack((r1, r2, r3))
    return R
# get_rotation_matrix(vpts, f, k)

In [79]:
def get_homography(...):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    # <YOUR CODE>
    pass

SyntaxError: invalid syntax (<ipython-input-79-1019841f9074>, line 1)

# Main function

In [67]:
im = np.asarray(Image.open('./eceb.jpg'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('./data.pickle'):
    with open('./data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)
    num_vpts = 3

In [70]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

num_vpts = 3
all_n, all_lines, all_centers = [], [], []

for i in range(num_vpts):
    print(f'Getting vanishing point {i}')
    
    # Get at least three lines from user input
    n_i, lines_i, centers_i = get_input_lines(im)
    all_n.append(n_i)
    all_lines.append(lines_i)
    all_centers.append(centers_i)

with open('data.pickle', 'wb') as f:
    pickle.dump([all_n, all_lines, all_centers], f)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
The delete and backspace keys act like right clicking
The enter key acts like middle clicking
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
The delete and backspace keys act like right clicking
The enter key acts like middle clicking
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (de

In [161]:
# Part (1)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    lines = np.asarray(all_lines[i])
    vpts[:, i] = get_vanishing_point(lines)
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')
print(vpts[:, 0], vpts[:, 1], vpts[:, 2])

[ 3.64725304e+03 -6.19815607e+01  1.00000000e+00] [2.12821831e+03 5.83624632e+03 1.00000000e+00] [-2.16018606e+03  3.67271545e+01  1.00000000e+00]


In [110]:
# Part (2) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts[:,0], vpts[:,2])
print(horizon_line)

# <YOUR CODE> Plot the ground horizon line
fig = plt.figure(); ax = fig.gca()
plot_horizon_line(ax, im, vpts[:,0], vpts[:,2])
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

[-0.01699449 -0.99985558  0.01059219]


In [158]:
# Part (3) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print(u, v, f, K)

2048.06544224965 1120.52389673485 -2334.13539207586 [[-2.33413539e+03  0.00000000e+00  2.04806544e+03]
 [ 0.00000000e+00 -2.33413539e+03  1.12052390e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [159]:
# Part (4) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, k)
print(R)

[[ 0.853122   -0.01523129 -0.52148908]
 [ 0.21971378 -0.89611959  0.3856106 ]
 [ 0.47318993  0.44355122  0.76115281]]
